<a href="https://colab.research.google.com/github/nohjuhyeon/study_data_analytics/blob/main/docs/quests/MLs/SpineSurgeryList_GridSearchCV_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df_SSL = pd.read_csv("SpineSurgeryList.csv")
df_SSL[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [2]:
df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '혈액형', '직업', '재발여부' ]]
# df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '직업', '재발여부' ]]
df_SSL_extract[:2]


,성별,신장,체중,흡연여부,연령,혈액형,직업,재발여부
0,2,163,60.3,0,66,RH+A,자영업,0
1,1,171,71.7,0,47,RH+A,운동선수,0


In [3]:
df_SSL_extract.isnull().sum()

성별        0
신장        0
체중        0
흡연여부      0
연령        0
혈액형       0
직업      415
재발여부      0
dtype: int64

## 전처리

In [4]:
df_SSL_extract.dropna(subset='직업',inplace=True)

<ipython-input-4-da3e4fb1ea5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SSL_extract.dropna(subset='직업',inplace=True)


In [5]:
target = df_SSL_extract['재발여부']
Xs = df_SSL_extract.drop(columns=['재발여부'])
features = pd.get_dummies(Xs) # 되도록이면 안쓰는 것이 좋음. / 컴퓨터가 바뀐 데이터를 이해를 못해서

In [6]:
features[:2]

,성별,신장,체중,흡연여부,연령,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,직업_건설업,...,직업_사무직,직업_사업가,직업_예술가,직업_운동선수,직업_운수업,직업_의료직,직업_자영업,직업_주부,직업_특수전문직,직업_학생
0,2,163,60.3,0,66,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,171,71.7,0,47,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## 특성공학 - Cross Validation

In [7]:
# 모델 : hyper parameters 교차 적용, 데이터셋 : 알맞게 섞어서 대입
# import 할 함수의 api를 검색하여  필요한 parameters 찾아서 사용하기
# 대신 parameters를 이해하기 위해서는 공식을 먼저 이해할 필요가 있음
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

hyper_parameters  = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}

estimator_model = DecisionTreeClassifier()
# grid_model = GridSearchCV(estimator_model,hyper_parameters)

# score 방식 지정
from sklearn.metrics import f1_score, make_scorer
scoring =  make_scorer(f1_score)
grid_model = GridSearchCV(estimator_model,hyper_parameters, scoring = scoring, cv=5) # 데이터셋 폴드

grid_model

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [8]:
grid_model.fit(features,target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [9]:
best_model = grid_model.best_estimator_
best_model

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=9, min_samples_leaf=8)

In [10]:
grid_model.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 9,
 'min_samples_leaf': 8,
 'min_samples_split': 2}

In [11]:
grid_model.scorer_

make_scorer(f1_score)

## 평가

In [12]:
from sklearn.metrics import classification_report
target_predict = best_model.predict(features)
print(classification_report(target,target_predict))

              precision    recall  f1-score   support

           0       0.99      0.62      0.76      1302
           1       0.25      0.93      0.40       177

    accuracy                           0.66      1479
   macro avg       0.62      0.78      0.58      1479
weighted avg       0.90      0.66      0.72      1479



In [13]:
target.value_counts()

0    1302
1     177
Name: 재발여부, dtype: int64

### Samplings

### Oversampling (많은 개수 맞춤)
- imblearn.over_sampling의 SMOTE 사용

In [16]:
from imblearn.over_sampling import SMOTE
overSampling = SMOTE(sampling_strategy=0.8)
feature_over_sample, target_over_sample= overSampling.fit_resample(features, target)
feature_over_sample.shape, target_over_sample.shape

((2343, 26), (2343,))

In [19]:
from collections import Counter
Counter(target_over_sample)

Counter({0: 1302, 1: 1041})

In [26]:
grid_model.fit(feature_over_sample,target_over_sample)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [27]:
oversampling_best_model = grid_model.best_estimator_
oversampling_best_model

DecisionTreeClassifier(class_weight='balanced', max_depth=9, min_samples_leaf=2,
                       min_samples_split=7)

### UnderSampling(적은 개수 맞춤)

In [21]:
# Oversampling (많은 개수 맞춤)
from imblearn.under_sampling import NearMiss
underSampling = NearMiss(sampling_strategy=0.8)
feature_under_sample, target_under_sample= underSampling.fit_resample(features, target)
feature_under_sample.shape, target_under_sample.shape

((398, 26), (398,))

In [22]:
Counter(target_under_sample)

Counter({0: 221, 1: 177})

In [28]:
grid_model.fit(feature_under_sample,target_under_sample)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [30]:
undersampling_best_model = grid_model.best_estimator_
undersampling_best_model

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=3, min_samples_leaf=9)

### Over와 Under 결합 Sampling

In [24]:
from imblearn.combine import SMOTEENN
combineSampling = SMOTEENN(sampling_strategy=0.5)
feature_combine_sample, target_combine_sample=combineSampling.fit_resample(features, target)
feature_combine_sample.shape, target_combine_sample.shape

((1188, 26), (1188,))

In [ ]:
Counter(target_combine_sample)

Counter({0: 50, 1: 25})

In [32]:
grid_model.fit(feature_combine_sample, target_combine_sample)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [33]:
combinesampling_best_model = grid_model.best_estimator_
combinesampling_best_model

DecisionTreeClassifier(max_depth=9, min_samples_leaf=4)

### sampling 별 F1 score 변화

In [36]:
# 기본
target_predict = best_model.predict(features)
print(classification_report(target,target_predict))

              precision    recall  f1-score   support

           0       0.99      0.62      0.76      1302
           1       0.25      0.93      0.40       177

    accuracy                           0.66      1479
   macro avg       0.62      0.78      0.58      1479
weighted avg       0.90      0.66      0.72      1479



In [37]:
# over sampling
target_over_predict = oversampling_best_model.predict(features)
print(classification_report(target,target_over_predict))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      1302
           1       0.55      0.64      0.59       177

    accuracy                           0.89      1479
   macro avg       0.75      0.78      0.76      1479
weighted avg       0.90      0.89      0.90      1479



In [38]:
# under sampling
target_under_predict = undersampling_best_model.predict(features)
print(classification_report(target,target_under_predict))

              precision    recall  f1-score   support

           0       0.91      0.61      0.73      1302
           1       0.17      0.58      0.26       177

    accuracy                           0.61      1479
   macro avg       0.54      0.59      0.49      1479
weighted avg       0.82      0.61      0.67      1479



In [39]:
# combine sampling
target_combine_predict = combinesampling_best_model.predict(features)
print(classification_report(target,target_combine_predict))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      1302
           1       0.46      0.41      0.43       177

    accuracy                           0.87      1479
   macro avg       0.69      0.67      0.68      1479
weighted avg       0.87      0.87      0.87      1479

